In [54]:
"""
HDM Experiments - Comparison with First-Order Methods

Comparing:
- GD: Gradient Descent
- Adam
- AdaGrad
- HDM: Hypergradient Descent Method

Test problems:
1. Quadratic Loss: ||Ax - b||^2
2. Quartic Loss: ||Ax - b||^4
3. Lp Loss: ||Ax - b||^p for p=2，4,6,8
4. Logistic Regression (unregularized) on MNIST

"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
import time


class QuadraticLoss:
    """f(x) = ||Ax - b||^2"""
    def __init__(self, A, b):
        self.A, self.b = A, b
        self.m, self.n = A.shape
        self.name = "Quadratic $\\|Ax-b\\|^2$"
        
    def __call__(self, x):
        r = self.A @ x - self.b
        return np.linalg.norm(r)**2
    
    def gradient(self, x):
        return 2 * self.A.T @ (self.A @ x - self.b)
    
    def optimal_value(self):
        x_opt, _, _, _ = np.linalg.lstsq(self.A, self.b, rcond=None)
        return self(x_opt)


class QuarticLoss:
    """f(x) = ||Ax - b||^4"""
    def __init__(self, A, b):
        self.A, self.b = A, b
        self.m, self.n = A.shape
        self.name = "Quartic $\\|Ax-b\\|^4$"
        
    def __call__(self, x):
        r = self.A @ x - self.b
        return np.linalg.norm(r)**4
    
    def gradient(self, x):
        r = self.A @ x - self.b
        return 4 * np.linalg.norm(r)**2 * (self.A.T @ r)
    
    def optimal_value(self):
        x_opt, _, _, _ = np.linalg.lstsq(self.A, self.b, rcond=None)
        return self(x_opt)


class LpLoss:
    r"""f(x) = ||Ax - b||_2^p"""
    def __init__(self, A, b, p):
        self.A, self.b, self.p = A, b, float(p)
        self.m, self.n = A.shape
        self.name = rf"$\|Ax-b\|^{int(p)}$"

    def __call__(self, x):
        r = self.A @ x - self.b
        return np.linalg.norm(r) ** self.p

    def gradient(self, x):
        r = self.A @ x - self.b
        nr = np.linalg.norm(r)
        if nr == 0.0:
            return np.zeros_like(x)

        # p = 1: grad ||r||_2 = A^T(r / ||r||_2)
        if np.isclose(self.p, 1.0):
            return self.A.T @ (r / nr)

        # general p > 1: grad = p * ||r||^{p-2} * A^T r
        return self.p * (nr ** (self.p - 2.0)) * (self.A.T @ r)

    def optimal_value(self):
        # Use least-squares solution as a common reference point
        x_opt, _, _, _ = np.linalg.lstsq(self.A, self.b, rcond=None)
        return self(x_opt)


class SelfBoundedLoss:
    """f(x,y) = e^(-x) + |y|^3"""
    def __init__(self):
        self.n = 2
        self.name = "Self-Bounded $e^{-x}+|y|^3$"
        
    def __call__(self, z):
        x, y = z[0], z[1]
        return np.exp(-x) + np.abs(y)**3
    
    def gradient(self, z):
        x, y = z[0], z[1]
        grad_y = 3 * y * np.abs(y) if y != 0 else 0.0
        return np.array([-np.exp(-x), grad_y])
    
    def optimal_value(self):
        return 0.0


class LogisticLoss:
    """Unregularized logistic regression"""
    def __init__(self, A, y, name="Logistic (MNIST)"):
        self.A, self.y = A, y
        self.m, self.n = A.shape
        self.name = name
        
    def sigmoid(self, z):
        return np.where(z >= 0, 1/(1+np.exp(-z)), np.exp(z)/(1+np.exp(z)))
    
    def __call__(self, x):
        z = self.y * (self.A @ x)
        loss = np.where(z >= 0, np.log1p(np.exp(-z)), -z + np.log1p(np.exp(z)))
        return np.mean(loss)
    
    def gradient(self, x):
        z = self.y * (self.A @ x)
        sig = self.sigmoid(-z)
        return -(1/self.m) * self.A.T @ (self.y * sig)
    
    def optimal_value(self):
        return 0.0
    
    def accuracy(self, x):
        pred = np.sign(self.A @ x)
        return np.mean(pred == self.y)


# =============================================================================
# OPTIMIZERS
# =============================================================================

class GD:
    """Gradient Descent"""
    def __init__(self, f, grad_f, alpha=0.01):
        self.f, self.grad_f, self.alpha = f, grad_f, alpha
        self.name = "GD"
        
    def optimize(self, x0, max_iter=2000):
        x = x0.copy()
        f_hist, g_hist, time_hist = [self.f(x)], [], [0.0]
        
        start_time = time.time()
        for k in range(max_iter):
            g = self.grad_f(x)
            g_hist.append(np.linalg.norm(g))
            x = x - self.alpha * g
            f_hist.append(self.f(x))
            time_hist.append(time.time() - start_time)
            
            if k % 500 == 0:
                print(f"    Iter {k}: f = {f_hist[-1]:.6e}, time = {time_hist[-1]:.2f}s")
        
        self.x_final = x
        self.total_time = time.time() - start_time
        return f_hist, g_hist, time_hist


class Adam:
    """Adam optimizer"""
    def __init__(self, f, grad_f, alpha=0.01, beta1=0.9, beta2=0.999, eps=1e-8):
        self.f, self.grad_f = f, grad_f
        self.alpha, self.beta1, self.beta2, self.eps = alpha, beta1, beta2, eps
        self.name = "Adam"
        
    def optimize(self, x0, max_iter=2000):
        x = x0.copy()
        m, v = np.zeros_like(x), np.zeros_like(x)
        f_hist, g_hist, time_hist = [self.f(x)], [], [0.0]
        
        start_time = time.time()
        for t in range(1, max_iter + 1):
            g = self.grad_f(x)
            g_hist.append(np.linalg.norm(g))
            m = self.beta1 * m + (1 - self.beta1) * g
            v = self.beta2 * v + (1 - self.beta2) * g**2
            m_hat = m / (1 - self.beta1**t)
            v_hat = v / (1 - self.beta2**t)
            x = x - self.alpha * m_hat / (np.sqrt(v_hat) + self.eps)
            f_hist.append(self.f(x))
            time_hist.append(time.time() - start_time)
            
            if t % 500 == 0:
                print(f"    Iter {t}: f = {f_hist[-1]:.6e}, time = {time_hist[-1]:.2f}s")
        
        self.x_final = x
        self.total_time = time.time() - start_time
        return f_hist, g_hist, time_hist


class AdaGrad:
    """AdaGrad optimizer"""
    def __init__(self, f, grad_f, alpha=0.1, eps=1e-8):
        self.f, self.grad_f = f, grad_f
        self.alpha, self.eps = alpha, eps
        self.name = "AdaGrad"
        
    def optimize(self, x0, max_iter=2000):
        x = x0.copy()
        G = np.zeros_like(x)
        f_hist, g_hist, time_hist = [self.f(x)], [], [0.0]
        
        start_time = time.time()
        for k in range(max_iter):
            g = self.grad_f(x)
            g_hist.append(np.linalg.norm(g))
            G += g**2
            x = x - self.alpha * g / (np.sqrt(G) + self.eps)
            f_hist.append(self.f(x))
            time_hist.append(time.time() - start_time)
            
            if k % 500 == 0:
                print(f"    Iter {k}: f = {f_hist[-1]:.6e}, time = {time_hist[-1]:.2f}s")
        
        self.x_final = x
        self.total_time = time.time() - start_time
        return f_hist, g_hist, time_hist


class HDM:
    """Hypergradient Descent Method with scalar stepsize (no upper bound)"""
    def __init__(self, f, grad_f, eta=0.1, alpha_min=1e-12):
        self.f, self.grad_f = f, grad_f
        self.eta, self.alpha_min = eta, alpha_min
        self.name = "OSGM-H"
        
    def optimize(self, x0, max_iter=2000, alpha0=0.01):
        x = x0.copy()
        alpha = alpha0
        f_hist, g_hist, alpha_hist, time_hist = [self.f(x)], [], [alpha], [0.0]
        
        start_time = time.time()
        for k in range(max_iter):
            g = self.grad_f(x)
            g_norm = np.linalg.norm(g)
            g_hist.append(g_norm)
            
            if g_norm < 1e-16:
                f_hist.append(f_hist[-1])
                alpha_hist.append(alpha)
                time_hist.append(time.time() - start_time)
                continue
            
            x_cand = x - alpha * g
            f_cand = self.f(x_cand)
            
            # Null step
            if f_cand < f_hist[-1]:
                x_new = x_cand
            else:
                x_new = x.copy()
            
            # Hypergradient update
            g_cand = self.grad_f(x_cand)
            hyperg = -np.dot(g_cand, g) / (g_norm**2 + 1e-16)
            alpha = max(alpha - self.eta * hyperg, self.alpha_min)
            
            x = x_new
            f_hist.append(self.f(x))
            alpha_hist.append(alpha)
            time_hist.append(time.time() - start_time)
            
            if k % 500 == 0:
                print(f"    Iter {k}: f = {f_hist[-1]:.6e}, α = {alpha:.2f}, time = {time_hist[-1]:.2f}s")
        
        self.alpha_hist = alpha_hist
        self.x_final = x
        self.total_time = time.time() - start_time
        return f_hist, g_hist, time_hist


# =============================================================================
# DATA LOADING
# =============================================================================

def create_regression_problem(m, n, cond, seed=42):
    np.random.seed(seed)
    U, _ = np.linalg.qr(np.random.randn(m, m))
    V, _ = np.linalg.qr(np.random.randn(n, n))
    s = np.logspace(0, -np.log10(cond), min(m, n))
    S = np.zeros((m, n))
    S[:min(m,n), :min(m,n)] = np.diag(s)
    A = U @ S @ V.T
    x_true = np.random.randn(n)
    b = A @ x_true + 0.01 * np.random.randn(m)
    return A, b


def load_mnist_data(n_samples=2000):
    """
    Load MNIST dataset. 
    sklearn caches the dataset after first download, so subsequent runs are fast.
    """
    print("Loading MNIST dataset...")
    mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
    X_all, y_all = mnist.data, mnist.target.astype(int)
    
    # Binary classification: 0 vs 1
    mask = (y_all == 0) | (y_all == 1)
    X = X_all[mask]
    y = y_all[mask]
    y = 2 * y - 1  # Convert to {-1, +1}
    
    print(f"  Full binary subset (0 vs 1): {len(y)} samples")
    
    # Subsample for faster experiments
    if n_samples is not None and n_samples < len(y):
        np.random.seed(42)
        idx = np.random.choice(len(y), n_samples, replace=False)
        X, y = X[idx], y[idx]
        print(f"  Using {n_samples} samples")
    
    X = X / 255.0
    A = StandardScaler().fit_transform(X)
    print(f"  Features: {A.shape[1]} (28x28 pixels)")
    print(f"  Class distribution: {np.sum(y==-1)} zeros, {np.sum(y==1)} ones")
    return A, y


# =============================================================================
# MAIN EXPERIMENT
# =============================================================================

def main():
    print("=" * 70)
    print("OSGM-H vs GD, Adam, AdaGrad (with MNIST)")
    print("=" * 70)
    
    # Problem setup
    m, n = 50, 20
    cond = 100
    max_iter = 2000
    
    A_reg, b_reg = create_regression_problem(m, n, cond)
    
    # Load MNIST
    A_log, y_log = load_mnist_data(n_samples=2000)  # Change to None for full dataset
    
    # Define problems with tuned hyperparameters
    problems = {
        'quadratic': {
            'loss': QuadraticLoss(A_reg, b_reg),
            'x0': np.zeros(n),
            'max_iter': max_iter,
            'params': {
                'GD': {'alpha': 0.1},
                'Adam': {'alpha': 0.1},
                'AdaGrad': {'alpha': 0.5},
                'OSGM-H': {'eta': 0.5, 'alpha0': 0.01}
            }
        },
        'quartic': {
            'loss': QuarticLoss(A_reg, b_reg),
            'x0': np.zeros(n),
            'max_iter': max_iter,
            'params': {
                'GD': {'alpha': 0.001},
                'Adam': {'alpha': 0.01},
                'AdaGrad': {'alpha': 0.1},
                'OSGM-H': {'eta': 0.1, 'alpha0': 0.0001}
            }
        },
        # 'lp1': {
        #     'loss': LpLoss(A_reg, b_reg, p=1.0),
        #     'x0': np.zeros(n),
        #     'max_iter': max_iter,
        #     'params': {
        #         'GD': {'alpha': 0.05},
        #         'Adam': {'alpha': 0.01},
        #         'AdaGrad': {'alpha': 0.2},
        #         'HDM': {'eta': 0.5, 'alpha0': 0.01},
        #     }
        # },
        'lp6': {
            'loss': LpLoss(A_reg, b_reg, p=6.0),
            'x0': np.zeros(n),
            'max_iter': max_iter,
            'params': {
                'GD': {'alpha': 1e-5},
                'Adam': {'alpha': 1e-3},
                'AdaGrad': {'alpha': 1e-2},
                'OSGM-H': {'eta': 0.1, 'alpha0': 1e-5},
            }
        },
        'lp8': {
            'loss': LpLoss(A_reg, b_reg, p=8.0),
            'x0': np.zeros(n),
            'max_iter': max_iter,
            'params': {
                'GD': {'alpha': 1e-6},
                'Adam': {'alpha': 5e-4},
                'AdaGrad': {'alpha': 5e-3},
                'OSGM-H': {'eta': 0.05, 'alpha0': 1e-6},
            }
        },
        # 'selfbounded': {
        #     'loss': SelfBoundedLoss(),
        #     'x0': np.array([0.0, 1.0]),
        #     'max_iter': max_iter,
        #     'params': {
        #         'GD': {'alpha': 0.3},
        #         'Adam': {'alpha': 0.3},
        #         'AdaGrad': {'alpha': 1.0},
        #         'HDM': {'eta': 0.5, 'alpha0': 0.1}
        #     }
        # },
        'logistic': {
            'loss': LogisticLoss(A_log, y_log, name="Logistic (MNIST)"),
            'x0': np.zeros(A_log.shape[1]),
            'max_iter': max_iter,
            'params': {
                'GD': {'alpha': 1.0},
                'Adam': {'alpha': 0.5},
                'AdaGrad': {'alpha': 2.0},
                'OSGM-H': {'eta': 55.0, 'alpha0': 1.0}
            }
        }
    }
    
    all_results = {}
    
    for prob_name, config in problems.items():
        print(f"\n{'='*50}")
        print(f"Problem: {config['loss'].name}")
        print(f"{'='*50}")
        
        loss = config['loss']
        x0 = config['x0']
        params = config['params']
        opt_val = loss.optimal_value()
        
        results = {}
        
        # GD
        print("  Running GD...")
        gd = GD(loss, loss.gradient, alpha=params['GD']['alpha'])
        f_hist, g_hist, time_hist = gd.optimize(x0.copy(), max_iter=config['max_iter'])
        results['GD'] = {'f_hist': f_hist, 'g_hist': g_hist, 'time_hist': time_hist,
                         'f_gap': np.maximum(np.array(f_hist) - opt_val, 1e-16),
                         'x_final': gd.x_final, 'total_time': gd.total_time}
        
        # Adam
        print("  Running Adam...")
        adam = Adam(loss, loss.gradient, alpha=params['Adam']['alpha'])
        f_hist, g_hist, time_hist = adam.optimize(x0.copy(), max_iter=config['max_iter'])
        results['Adam'] = {'f_hist': f_hist, 'g_hist': g_hist, 'time_hist': time_hist,
                           'f_gap': np.maximum(np.array(f_hist) - opt_val, 1e-16),
                           'x_final': adam.x_final, 'total_time': adam.total_time}
        
        # AdaGrad
        print("  Running AdaGrad...")
        adagrad = AdaGrad(loss, loss.gradient, alpha=params['AdaGrad']['alpha'])
        f_hist, g_hist, time_hist = adagrad.optimize(x0.copy(), max_iter=config['max_iter'])
        results['AdaGrad'] = {'f_hist': f_hist, 'g_hist': g_hist, 'time_hist': time_hist,
                              'f_gap': np.maximum(np.array(f_hist) - opt_val, 1e-16),
                              'x_final': adagrad.x_final, 'total_time': adagrad.total_time}
        
        # HDM
        print("  Running OSGM-H...")
        hdm = HDM(loss, loss.gradient, eta=params['OSGM-H']['eta'])
        f_hist, g_hist, time_hist = hdm.optimize(x0.copy(), max_iter=config['max_iter'], 
                                                  alpha0=params['OSGM-H']['alpha0'])
        results['OSGM-H'] = {'f_hist': f_hist, 'g_hist': g_hist, 'time_hist': time_hist,
                          'f_gap': np.maximum(np.array(f_hist) - opt_val, 1e-16),
                          'alpha_hist': hdm.alpha_hist,
                          'x_final': hdm.x_final, 'total_time': hdm.total_time}
        
        all_results[prob_name] = {'results': results, 'config': config}
    
    # ==========================================================================
    # PLOTTING
    # ==========================================================================
    
    colors = {
        'GD': '#1f77b4',
        'Adam': '#d62728',
        'AdaGrad': '#9467bd',
        'OSGM-H': '#2ca02c',
    }
    
    # Order and titles for all problems
    prob_order = ['quadratic', 'quartic', 'lp6', 'lp8', 'logistic']
    titles = [
        '$\\|Ax-b\\|^2$',
        '$\\|Ax-b\\|^4$',
        '$\\|Ax-b\\|^6$',
        '$\\|Ax-b\\|^8$',
        'Logistic (MNIST)',
    ]
    n_probs = len(prob_order)
    
    # Main comparison figure (2 x n_probs) - by iteration
    fig, axes = plt.subplots(2, n_probs, figsize=(4 * n_probs, 8))
    
    for col, prob_name in enumerate(prob_order):
        title = titles[col]
        results = all_results[prob_name]['results']
        max_iter_prob = all_results[prob_name]['config']['max_iter']
        
        # Function value gap (top row)
        ax = axes[0, col]
        for name in ['GD', 'Adam', 'AdaGrad', 'OSGM-H']:
            res = results[name]
            ax.semilogy(res['f_gap'], label=name, color=colors[name], 
                       linewidth=2, alpha=0.9)
        ax.set_xlabel('Iteration')
        if col == 0:
            ax.set_ylabel('Function value gap')
        ax.set_title(title)
        ax.grid(True, alpha=0.3)
        ax.set_xlim([0, max_iter_prob])
        
        # Gradient norm (bottom row)
        ax = axes[1, col]
        for name in ['GD', 'Adam', 'AdaGrad', 'OSGM-H']:
            res = results[name]
            ax.semilogy(res['g_hist'], label=name, color=colors[name],
                       linewidth=2, alpha=0.9)
        ax.set_xlabel('Iteration')
        if col == 0:
            ax.set_ylabel('Gradient Norm')
        ax.grid(True, alpha=0.3)
        ax.set_xlim([0, max_iter_prob])
    
    # Legend
    handles, labels = axes[0, 0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=4, fontsize=11,
               bbox_to_anchor=(0.5, -0.02))
    
    plt.suptitle('OSGM-H vs First-Order Methods (by Iteration)', fontsize=14, fontweight='bold')
    plt.tight_layout(rect=[0, 0.05, 1, 0.96])
    plt.savefig('hdm_comparison_iter.png', dpi=150, bbox_inches='tight')
    plt.close()
    
    # Comparison by wall-clock time (2 x n_probs)
    fig, axes = plt.subplots(2, n_probs, figsize=(4 * n_probs, 8))
    
    for col, prob_name in enumerate(prob_order):
        title = titles[col]
        results = all_results[prob_name]['results']
        
        # Function value gap vs time (top row)
        ax = axes[0, col]
        for name in ['GD', 'Adam', 'AdaGrad', 'OSGM-H']:
            res = results[name]
            ax.semilogy(res['time_hist'], res['f_gap'], label=name, color=colors[name], 
                       linewidth=2, alpha=0.9)
        ax.set_xlabel('Time (s)')
        if col == 0:
            ax.set_ylabel('Function value gap')
        ax.set_title(title)
        ax.grid(True, alpha=0.3)
        
        # Gradient norm vs time (bottom row)
        ax = axes[1, col]
        for name in ['GD', 'Adam', 'AdaGrad', 'OSGM-H']:
            res = results[name]
            # time_hist has one more element than g_hist
            ax.semilogy(res['time_hist'][1:], res['g_hist'], label=name, color=colors[name],
                       linewidth=2, alpha=0.9)
        ax.set_xlabel('Time (s)')
        if col == 0:
            ax.set_ylabel('Gradient Norm')
        ax.grid(True, alpha=0.3)
    
    # Legend
    handles, labels = axes[0, 0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=4, fontsize=11,
               bbox_to_anchor=(0.5, -0.02))
    
    plt.suptitle('OSGM-H vs First-Order Methods (by Wall-Clock Time)', fontsize=14, fontweight='bold')
    plt.tight_layout(rect=[0, 0.05, 1, 0.96])
    plt.savefig('hdm_comparison_time.png', dpi=150, bbox_inches='tight')
    plt.close()
    
    # ==========================================================================
    # Results Table
    # ==========================================================================
    print("\n" + "=" * 70)
    print(f"FINAL RESULTS TABLE (after {max_iter} iterations)")
    print("=" * 70)
    
    print(f"\n{'Algorithm':<12}", end="")
    for title in titles:
        clean_title = title.replace('$', '').replace('\\|', '|').replace('\\', '')
        if len(clean_title) > 16:
            clean_title = clean_title[:14] + ".."
        print(f"{clean_title:<18}", end="")
    print()
    print("-" * 84)
    
    print("\nOptimality Gap:")
    for method in ['GD', 'Adam', 'AdaGrad', 'OSGM-H']:
        print(f"{method:<12}", end="")
        for prob_name in prob_order:
            results = all_results[prob_name]['results']
            final_gap = results[method]['f_gap'][-1]
            print(f"{final_gap:<18.2e}", end="")
        print()
    
    print("\nRunning Time (seconds):")
    for method in ['GD', 'Adam', 'AdaGrad', 'OSGM-H']:
        print(f"{method:<12}", end="")
        for prob_name in prob_order:
            results = all_results[prob_name]['results']
            total_time = results[method]['total_time']
            print(f"{total_time:<18.3f}", end="")
        print()
    
    # Print logistic regression accuracy
    print("\n" + "-" * 50)
    print("Logistic Regression (MNIST) Training Accuracy:")
    print("-" * 50)
    loss_log = all_results['logistic']['config']['loss']
    for method in ['GD', 'Adam', 'AdaGrad', 'OSGM-H']:
        x_final = all_results['logistic']['results'][method]['x_final']
        acc = loss_log.accuracy(x_final) * 100
        total_time = all_results['logistic']['results'][method]['total_time']
        print(f"  {method}: {acc:.2f}% (time: {total_time:.3f}s)")
    
    # ==========================================================================
    # Individual detailed plots with stepsize
    # ==========================================================================
    
    for prob_name, title in zip(prob_order, titles):
        results = all_results[prob_name]['results']
        config = all_results[prob_name]['config']
        
        fig, axes = plt.subplots(1, 3, figsize=(14, 4))
        
        # Function value gap
        ax = axes[0]
        for name in ['GD', 'Adam', 'AdaGrad', 'OSGM-H']:
            res = results[name]
            ax.semilogy(res['f_gap'], label=name, color=colors[name], linewidth=2)
        ax.set_xlabel('Iteration')
        ax.set_ylabel('Function value gap')
        ax.set_title('Function Value Gap')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Gradient norm
        ax = axes[1]
        for name in ['GD', 'Adam', 'AdaGrad', 'OSGM-H']:
            res = results[name]
            ax.semilogy(res['g_hist'], label=name, color=colors[name], linewidth=2)
        ax.set_xlabel('Iteration')
        ax.set_ylabel('Gradient Norm')
        ax.set_title('Gradient Norm')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # HDM stepsize
        ax = axes[2]
        ax.semilogy(results['OSGM-H']['alpha_hist'], color=colors['OSGM-H'], linewidth=2)
        ax.set_xlabel('Iteration')
        ax.set_ylabel('Stepsize $\\alpha$')
        ax.set_title('OSGM-H Learned Stepsize')
        ax.grid(True, alpha=0.3)
        
        plt.suptitle(f'{title}', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(f'hdm_{prob_name}_detailed.png', dpi=150, bbox_inches='tight')
        plt.close()
    
    # ==========================================================================
    # Summary timing table
    # ==========================================================================
    print("\n" + "=" * 70)
    print("TIMING SUMMARY")
    print("=" * 70)
    
    print(f"\n{'Problem':<20} {'GD':<12} {'Adam':<12} {'AdaGrad':<12} {'OSGM-H':<12}")
    print("-" * 68)
    for prob_name, title in zip(prob_order, titles):
        results = all_results[prob_name]['results']
        clean_title = title.replace('$', '').replace('\\|', '|').replace('\\', '')[:18]
        print(f"{clean_title:<20}", end="")
        for method in ['GD', 'Adam', 'AdaGrad', 'OSGM-H']:
            t = results[method]['total_time']
            print(f"{t:<12.3f}", end="")
        print()
    
    print("\nNote: HDM requires 2 gradient evaluations per iteration (vs 1 for others)")
    
    print("\n" + "=" * 70)
    print("Plots saved:")
    print("  - hdm_comparison_iter.png  (convergence by iteration)")
    print("  - hdm_comparison_time.png  (convergence by wall-clock time)")
    for prob_name in prob_order:
        print(f"  - hdm_{prob_name}_detailed.png")
    print("=" * 70)


if __name__ == "__main__":
    main()


OSGM-H vs GD, Adam, AdaGrad (with MNIST)
Loading MNIST dataset...
  Full binary subset (0 vs 1): 14780 samples
  Using 2000 samples
  Features: 784 (28x28 pixels)
  Class distribution: 961 zeros, 1039 ones

Problem: Quadratic $\|Ax-b\|^2$
  Running GD...
    Iter 0: f = 7.768161e-01, time = 0.00s
    Iter 500: f = 1.564778e-02, time = 0.01s
    Iter 1000: f = 1.120617e-02, time = 0.01s
    Iter 1500: f = 8.951422e-03, time = 0.02s
  Running Adam...
    Iter 500: f = 2.148593e-03, time = 0.01s
    Iter 1000: f = 1.864089e-03, time = 0.02s
    Iter 1500: f = 1.836492e-03, time = 0.03s
    Iter 2000: f = 1.833833e-03, time = 0.04s
  Running AdaGrad...
    Iter 0: f = 1.325292e+00, time = 0.00s
    Iter 500: f = 3.995661e-03, time = 0.01s
    Iter 1000: f = 2.520077e-03, time = 0.01s
    Iter 1500: f = 2.120709e-03, time = 0.02s
  Running OSGM-H...
    Iter 0: f = 1.039531e+00, α = 0.50, time = 0.00s
    Iter 500: f = 4.159829e-03, α = 1.81, time = 0.01s
    Iter 1000: f = 2.775500e-03, α 

/var/folders/3j/9b_qh0z13pv__zzfxvd58x_r0000gn/T/ipykernel_6524/1487973385.py:125: RuntimeWarning: overflow encountered in exp
  loss = np.where(z >= 0, np.log1p(np.exp(-z)), -z + np.log1p(np.exp(z)))
/var/folders/3j/9b_qh0z13pv__zzfxvd58x_r0000gn/T/ipykernel_6524/1487973385.py:121: RuntimeWarning: overflow encountered in exp
  return np.where(z >= 0, 1/(1+np.exp(-z)), np.exp(z)/(1+np.exp(z)))


    Iter 500: f = 7.323055e-09, time = 0.80s
    Iter 1000: f = 2.502926e-09, time = 1.57s
    Iter 1500: f = 1.265877e-09, time = 2.34s
    Iter 2000: f = 7.530311e-10, time = 3.11s
  Running AdaGrad...
    Iter 0: f = 9.201191e-01, time = 0.00s
    Iter 500: f = 8.879922e-07, time = 0.79s
    Iter 1000: f = 4.426741e-07, time = 1.58s
    Iter 1500: f = 2.955970e-07, time = 2.36s
  Running OSGM-H...
    Iter 0: f = 5.553712e-02, α = 0.87, time = 0.00s
    Iter 500: f = 4.687950e-09, α = 25700.56, time = 1.51s
    Iter 1000: f = 1.559815e-09, α = 42204.48, time = 3.02s
    Iter 1500: f = 8.530036e-10, α = 48593.40, time = 4.54s

FINAL RESULTS TABLE (after 2000 iterations)

Algorithm   |Ax-b|^2          |Ax-b|^4          |Ax-b|^6          |Ax-b|^8          Logistic (MNIST)  
------------------------------------------------------------------------------------

Optimality Gap:
GD          5.74e-03          2.16e-02          7.36e-01          1.18e+00          9.28e-06          
Adam      